In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
from mpisppy.opt.lshaped import LShapedMethod
from pyomo.environ import value as pyoval
import matplotlib.pyplot as plt
from matplotlib import rc

[    0.00] Initializing mpi-sppy


In [2]:
def build_model(yields):
    model = ConcreteModel()

    # Variables
    model.X = Var(["WHEAT", "CORN", "BEETS"], within=NonNegativeReals)
    model.Y = Var(["WHEAT", "CORN"], within=NonNegativeReals)
    model.W = Var(
        ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"],
        within=NonNegativeReals,
    )

    # Objective function
    # model.WHEAT_COST = 150 * model.X["WHEAT"]
    # model.CORN_COST = 230 * model.X["CORN"]
    # model.BEETS_COST = 260 * model.X["BEETS"]
    # model.PLANTING_COST = model.WHEAT_COST + model.CORN_COST + model.BEETS_COST
    model.PLANTING_COST = 150 * model.X["WHEAT"] + 230 * model.X["CORN"] + 260 * model.X["BEETS"]
    model.PURCHASE_COST = 238 * model.Y["WHEAT"] + 210 * model.Y["CORN"]
    model.SALES_REVENUE = (
        170 * model.W["WHEAT"] + 150 * model.W["CORN"]
        + 36 * model.W["BEETS_FAVORABLE"] + 10 * model.W["BEETS_UNFAVORABLE"]
    )
    model.OBJ = Objective(
        expr=model.PLANTING_COST + model.PURCHASE_COST - model.SALES_REVENUE,
        sense=minimize
    )

    # Constraints
    model.CONSTR= ConstraintList()

    model.CONSTR.add(summation(model.X) <= 500)
    model.CONSTR.add(
        yields[0] * model.X["WHEAT"] + model.Y["WHEAT"] - model.W["WHEAT"] >= 200
    )
    model.CONSTR.add(
        yields[1] * model.X["CORN"] + model.Y["CORN"] - model.W["CORN"] >= 240
    )
    model.CONSTR.add(
        yields[2] * model.X["BEETS"] - model.W["BEETS_FAVORABLE"] - model.W["BEETS_UNFAVORABLE"] >= 0
    )
    model.W["BEETS_FAVORABLE"].setub(6000)

    return model

In [3]:

def scenario_creator(scenario_name):
    if scenario_name == "good":
        yields = [3, 3.6, 24]
    elif scenario_name == "average":
        yields = [2.5, 3, 20]
    elif scenario_name == "bad":
        yields = [2, 2.4, 16]
    else:
        raise ValueError("Unrecognized scenario name")

    scenario_probabilities = {'good': 0.33, 'average': 0.33, 'bad':0.33} 
    model = build_model(yields)
    sputils.attach_root_node(model, model.PLANTING_COST, [model.X])
    model._mpisppy_probability = 1.0/3
    # model.pprint()
    return model

In [5]:
all_scenario_names = ["good", "average", "bad"]
bounds = {name: -432000 for name in all_scenario_names}
options = {
    "root_solver": "gurobi",
    "sp_solver": "gurobi",
    "sp_solver_options" : {"threads" : 1},
    # "valid_eta_lb": bounds,
    "max_iter": 10,
    # 'verbose': True,
}

ls = LShapedMethod(options, all_scenario_names, scenario_creator)
result = ls.lshaped_algorithm()

variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

[    0.43] Initializing SPBase
obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 150*X[WHEAT] + 230*X[CORN] + 260*X[BEETS] + eta[good] + eta[average] + eta[bad]
Current Iteration: 1, 	 Time Elapsed: 0.00, 	 Current Objective: -Inf
Current Iteration: 2, 	 Time Elapsed: 0.90, 	 Time Spent on Last Master: 0.23, 	 Time Spent Generating Last Cut: 0.68, 	 Number of cuts added: 3, 	 Current Objective: -231100.00
Current Iteration: 3, 	 Time Elapsed: 1.84, 	 Time Spent on Last Master: 0.22, 	 Time Spent Generating Last Cut: 0.72, 	 Number of cuts added: 3, 	 Current Objective: -119614.69
Current Iteration: 4, 	 Time Elapsed: 2.84, 	 Time Spent on Last Master: 0.28, 	 Time Spent Generating Last Cut: 0.71, 	 Number of cuts added: 1, 	 Current Objective: -110249.63
Current Iteration: 5, 	 Time Elapsed: 3.81, 	 Time Spent on Last Master: 0.24, 	 Time Spent Generating Last Cut: 0.72, 	 Number of cuts added: 1, 	 Current Objective: -108483

In [6]:
ls.root.X.pprint()

X : Size=3, Index={WHEAT, CORN, BEETS}
    Key   : Lower : Value              : Upper : Fixed : Stale : Domain
    BEETS :     0 : 250.00000000000003 :  None : False :  True : NonNegativeReals
     CORN :     0 :  80.00000000000027 :  None : False :  True : NonNegativeReals
    WHEAT :     0 : 169.99999999999966 :  None : False :  True : NonNegativeReals


In [7]:
ls.root.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 150*X[WHEAT] + 230*X[CORN] + 260*X[BEETS] + eta[good] + eta[average] + eta[bad]


In [9]:
variables

{('good', 'X[BEETS]'): 250.00000000000003,
 ('good', 'X[CORN]'): 80.00000000000027,
 ('good', 'X[WHEAT]'): 169.99999999999966,
 ('average', 'X[BEETS]'): 250.00000000000003,
 ('average', 'X[CORN]'): 80.00000000000027,
 ('average', 'X[WHEAT]'): 169.99999999999966,
 ('bad', 'X[BEETS]'): 250.00000000000003,
 ('bad', 'X[CORN]'): 80.00000000000027,
 ('bad', 'X[WHEAT]'): 169.99999999999966}